In [ ]:
# Kora S. Hughes - Artificial Intelligence Project 1: 11 Puzzel Problem with A*

In [105]:
""" PROJECT NOTES
1-2 students per project
“Bramble system” → only 1 tile moves at a time

standard:
 -11 tile problem
 -3 X 4 board
"""

import random
import copy

GOAL_BOARD = [0,1,2,3,4,5,6,7,8,9,10,11]  # [[0,1,2,3],[4,5,6,7],[8,9,10,11]]
action_translation= {1:"U", 2:"D", 3:"L", 4:"R"}  # output translator
action_translation2= {1:"up", 2:"down", 3:"left", 4:"right"}  # testing helper

In [106]:
class Puzzle:
    '''
    Main Puzzle Class:
    - contains board and helper functions to navigate board
    '''
    def __init__(self, x=4, y=3):  # init with board dimensions
        assert type(x) == int and type(y) == int
        temp_board = []
        for i in range(y):  # circumvents shallow copy issue with [[0]*x]*y
            temp_row = []
            for i in range(x):
                temp_row.append(0)
            temp_board.append(temp_row)
        self.board = copy.deepcopy(temp_board)  # main board storage
        # stored vars for convenience
        self.size = (x*y) - 1
        self.dimensions = (x, y)

    def __bool__(self):  # validating board
        nums = []
        for y in self.board:  # is valid if all unique nums within size
            for x in y:
                assert type(x) == int
                if x in nums or x < 0 or x > self.size:
                    return False
                else:
                    nums.append(x)
        return True
    
    def __eq__(self, rhs):  # compare the position of all numbers on the board
        if self.dimensions == rhs.dimensions:
            for i in range(self.dimensions[1]):
                for j in range(self.dimensions[0]):
                    if not self.board[i][j] == rhs.board[i][j]:
                        return False
            return True
        else:
            return False

    def show(self):  # show the board for testing
        print("*BOARD* " + str(self.size) + "-Puzzle: " + str(self.dimensions))
        for y in self.board:
            out = ""
            for x in y:
                out += " " + str(x)
                if x < 10:
                    out += " "
            print("[ " + out + " ]")

    def random_fill(self):  # randomizer for board values in testing
        nums = random.sample(range(0, self.size+1), self.size+1)
        # print("adding random board:", nums)
        self.fill(nums)

    def fill(self, vals):  # manually fill puzzle
        '''
        :param vals: 1d array of puzzle values
        fills table
        '''
        assert len(vals) == self.size+1
        i = 0
        for y in range(0, self.dimensions[1]):
            for x in range(0, self.dimensions[0]):
                # print(vals[i], ":", x, ",", y, ",", i)
                self.board[y][x] = vals[i]
                i += 1
    
    def h(self):
        '''
        Manhattan Distance heuristic function for a valid board
        '''
        ideal_board = Puzzle()
        ideal_board.fill(GOAL_BOARD)
        assert bool(self)
        assert bool(ideal_board)
        h_sum = 0
        
        for i in range(1, self.size+1):
            ideal_coords = ideal_board.get_num(i)
            curr_coords = self.get_num(i)
            # make max for chessboar distance
            value = abs(ideal_coords[0]-curr_coords[0]) + abs(ideal_coords[1]-curr_coords[1]) 
            h_sum += value
        return h_sum
    
    def action(self, act):
        '''
        takes integer input representing a puzzle move
        edits the present board according to said move'''
        assert type(act) == int
        assert act <= 4 and act >= 1
        zero_place = self.get_num(0)  # y,x of 0's place
        assert zero_place[0] != -1 and zero_place [1] != -1
            
        if act == 1:  # up
            if not zero_place[0] == self.dimensions[1]-1:  # cant move up if there is no tile under it
                move_tile = self.board[zero_place[0]+1][zero_place[1]]
                self.board[zero_place[0]+1][zero_place[1]] = 0
                self.board[zero_place[0]][zero_place[1]] = move_tile
        elif act == 2:  # down
            if not zero_place[0] == 0:  # cant move down if there is no tile above it
                move_tile = self.board[zero_place[0]-1][zero_place[1]]
                self.board[zero_place[0]-1][zero_place[1]] = 0
                self.board[zero_place[0]][zero_place[1]] = move_tile
        elif act == 3:  # left
            if not zero_place[1] == self.dimensions[0]-1:  # cant move left if theres no tile to the right of it
                move_tile = self.board[zero_place[0]][zero_place[1]+1]
                self.board[zero_place[0]][zero_place[1]+1] = 0
                self.board[zero_place[0]][zero_place[1]] = move_tile
        elif act == 4:  # right
            if not zero_place[1] == 0:  # cant move right if there is no tile to the left of it
                move_tile = self.board[zero_place[0]][zero_place[1]-1]
                self.board[zero_place[0]][zero_place[1]-1] = 0
                self.board[zero_place[0]][zero_place[1]] = move_tile
            
        else:
            print("serious problem with action script")
    
    def get_num(self, num):  # helper that finds the (index) position of a number within this board
        for y in range(self.dimensions[1]):
            for x in range(self.dimensions[0]):
                if self.board[y][x] == num:
                    return (y,x)
        return (-1,-1)
    
    def copy(self):  # deep-copy function
        p1 = Puzzle(self.dimensions[0], self.dimensions[1])
        p1.board = copy.deepcopy(self.board)
        return p1


In [114]:
class State:
    ''' helper class to represent nodes in A*'''
    def __init__(self, puzz, move_num=0):
        self.puzz = puzz
        self.move = move_num
        
    def show(self):  # show node
        print("\nMove #:", self.move)
        print("f =", self.f(), "h =", self.puzz.h())
        self.puzz.show()
    
    def __eq__(self, rhs):  # compare board states
        if isinstance(rhs, State):
            return self.puzz == rhs.puzz  # and self.move == rhs.move
        return False
    
    def f(self, weight=1.0):  # weight = 1.2?, weight = 1.4?
        return self.move + weight*self.puzz.h()
        
def a_star(p1, weight=1.0):
    '''
    Main Algorithm: A*
    uses manhattan distance heuristic h(n) 
    and path cost, aka sum(moves), g(n) 
    to search for puzzle solution
    '''
    node = State(p1, 0)
    # frontier = [new_node for new_node in expand(node)]
    frontier = [(node, 0)]
    reach = []  # no repeats
    
    depth = -1  # -1 counteracts the popping of the root node so the final depth starts at 0
    num_nodes = 0
    action_sequence = []
    f_sequence = []
    
    while len(frontier) != 0:
        node_info = frontier.pop()
        node = node_info[0]
        if node_info[1] != 0:
            action_sequence.append(action_translation[node_info[1]])
        f_sequence.append(node.f())
        node.show()
        depth += 1
        if node.puzz.h() == 0:  # checking for goal node
            print("goal!!!")
            return node, depth, num_nodes, action_sequence, f_sequence
        for new_state, action in expand(node):
            num_nodes += 1
            n_ind = -1  # index of potential node in reach
            for r in range(len(reach)):
                if reach[r] == new_state:
                    n_ind = r
            if n_ind == -1:  # double check this works
                j = len(frontier)-1
                while j >= 0 and frontier[j][0].f(weight) <= new_state.f(weight):  # insertion sort for priority queue
                    j -= 1
                add_state = (new_state, action)  # needed structure so we can save an action for the output sequence
                if j == len(frontier)-1:  # edge case with inserting to the end of a list
                    frontier.append(add_state)
                else:
                    frontier.insert(j, add_state)
                reach.append(new_state)
            else:
                reach[n_ind].move = min(reach[n_ind].move, new_state.move)  # saves path with smallest move number
                # print("passed repeat state")
    node.show()
    return node  # if goal node isnt found then we just return the last node we looked at...?

def expand(curr_state):
    ''' returns a list of children of the current state ordered by f()'''
    lst = []
    for i in range(1, 5):
        temp_puzz1 = curr_state.puzz.copy()
        temp_puzz1.action(i)  # make a puzzle copy with the new action done
        new_state = State(temp_puzz1, curr_state.move+1)
        # print("Move:", action_translation[i])
        if curr_state != new_state:
            yield new_state, i

In [116]:
if __name__ == '__main__':
    print("start...\n")

    start_board = Puzzle()
    goal_board = Puzzle()
    weights = [1.0, 1.2, 1.4]
    weight = weights[random.randint(0,len(weights)-1)]
    print("Weight:", weight)
    
    start_board.fill([1,2,7,6,4,5,3,11,8,0,9,10])
    # TODO: read in initial list into start_board and the ideal board sequence into GOAL_BOARD
    
    assert bool(start_board)
    solution_node, depth, num_nodes, actions, f_sequence = a_star(start_board, weight)
    assert bool(solution_node)
    print("Depth:", depth)
    print("# Nodes Generated:", num_nodes)
    print("Actions:", actions)
    print("F's:", f_sequence)

    print("\nend...")

start...

Weight: 1.2

Move #: 0
f = 11.0 h = 11
*BOARD* 11-Puzzle: (4, 3)
[  1  2  7  6  ]
[  4  5  3  11 ]
[  8  0  9  10 ]

Move #: 1
f = 11.0 h = 10
*BOARD* 11-Puzzle: (4, 3)
[  1  2  7  6  ]
[  4  5  3  11 ]
[  8  9  0  10 ]

Move #: 2
f = 11.0 h = 9
*BOARD* 11-Puzzle: (4, 3)
[  1  2  7  6  ]
[  4  5  3  11 ]
[  8  9  10 0  ]

Move #: 3
f = 11.0 h = 8
*BOARD* 11-Puzzle: (4, 3)
[  1  2  7  6  ]
[  4  5  3  0  ]
[  8  9  10 11 ]

Move #: 4
f = 11.0 h = 7
*BOARD* 11-Puzzle: (4, 3)
[  1  2  7  0  ]
[  4  5  3  6  ]
[  8  9  10 11 ]

Move #: 5
f = 11.0 h = 6
*BOARD* 11-Puzzle: (4, 3)
[  1  2  0  7  ]
[  4  5  3  6  ]
[  8  9  10 11 ]

Move #: 6
f = 11.0 h = 5
*BOARD* 11-Puzzle: (4, 3)
[  1  2  3  7  ]
[  4  5  0  6  ]
[  8  9  10 11 ]

Move #: 7
f = 11.0 h = 4
*BOARD* 11-Puzzle: (4, 3)
[  1  2  3  7  ]
[  4  5  6  0  ]
[  8  9  10 11 ]

Move #: 8
f = 11.0 h = 3
*BOARD* 11-Puzzle: (4, 3)
[  1  2  3  0  ]
[  4  5  6  7  ]
[  8  9  10 11 ]

Move #: 9
f = 11.0 h = 2
*BOARD* 11-Puzzle: (4, 